In [16]:
import requests
import numpy as np
import datetime

Set up your API token from your profile on observation portal

In [28]:
API_TOKEN   = 'XXX'
PROPOSAL_ID = 'LCOEPO2019A-001' 
OBSERVATION_TITLE   = 'StudentData_Target_E'
TARGET_NAME         = 'TARGET_E' # Target Period: 7.6 Hours
RA                  = 45.06433
DEC                 = -4.99483
OBSERVATION_CADENCE = 21./60 # hours
START_TIME          = '2019-10-30 20:00:00'
END_TIME            = '2019-10-31 05:00:00'
EXPOSURE_TIME       = 90 # seconds
FILTER1             = 'v'
FILTER2             = 'b'
print('The observation cadence is: ',OBSERVATION_CADENCE,' Hours, or ',OBSERVATION_CADENCE*60,' minutes')

The observation cadence is:  0.35  Hours, or  21.0  minutes


In [29]:
# Determine when the observations are to take place
# The period is NOT the period of the variable. 
# The period is the cadence at which you want to observe, in HOURS
# The Jitter is the window within which your observation may take place. 

# Want jitter to equal period to "fully cover" time
jitter = OBSERVATION_CADENCE

cadence = {
    'start': START_TIME,
    'end': END_TIME,
    'period': OBSERVATION_CADENCE,
    'jitter': jitter
}

In [30]:
target = {
    'name': TARGET_NAME,
    'type': 'ICRS',
    'ra': RA,
    'dec': DEC,
    'epoch': 2000
}

In [31]:
# Keep Max_airmass at 1.6
# min_lunar_distance is in DEGREES, keep at 20 degrees

constraints = {
    'max_airmass': 2.0,
    'min_lunar_distance': 20
}

In [32]:
# This does not need to be changed.
# Exposure time is in SECONDS

configurations = [
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': EXPOSURE_TIME,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': FILTER1
                }
            }
        ]
    },
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': EXPOSURE_TIME,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': FILTER2
                }
            }
        ]
    }
]

In [33]:
# This cell does not need to be edited.

# We do not provide windows for a cadence request
windows = []

# The telescope class that should be used for this observation
location = {
    'telescope_class': '0m4'
}

In [34]:
# Please input the name of your observation request. Eg. Group_Star_Target_B
requestgroup = {
    'name': OBSERVATION_TITLE,  # The title
    'proposal': PROPOSAL_ID,
    'ipp_value': 1.05,
    'operator': 'SINGLE',
    'observation_type': 'NORMAL',
    'requests': [{
        'cadence': cadence,
        'configurations': configurations,
        'windows': windows,
        'location': location,
    }]
}

In [35]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/cadence/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=requestgroup  # Make sure you use json!
)

In [36]:
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

In [37]:
cadence_request = response.json()

print('Cadence generated {} requests'.format(len(cadence_request['requests'])))
i = 1
for request in cadence_request['requests']:
    print('Request {0} window start: {1} window end: {2}'.format(
        i, request['windows'][0]['start'], request['windows'][0]['end']
    ))
    i = i + 1

Cadence generated 26 requests
Request 1 window start: 2019-10-30T20:00:00Z window end: 2019-10-30T20:10:30Z
Request 2 window start: 2019-10-30T20:10:30Z window end: 2019-10-30T20:31:30Z
Request 3 window start: 2019-10-30T20:31:30Z window end: 2019-10-30T20:52:30Z
Request 4 window start: 2019-10-30T20:52:30Z window end: 2019-10-30T21:13:30Z
Request 5 window start: 2019-10-30T21:13:30Z window end: 2019-10-30T21:34:30Z
Request 6 window start: 2019-10-30T21:34:30Z window end: 2019-10-30T21:55:30Z
Request 7 window start: 2019-10-30T21:55:30Z window end: 2019-10-30T22:16:30Z
Request 8 window start: 2019-10-30T22:16:30Z window end: 2019-10-30T22:37:30Z
Request 9 window start: 2019-10-30T22:37:30Z window end: 2019-10-30T22:58:30Z
Request 10 window start: 2019-10-30T22:58:30Z window end: 2019-10-30T23:19:30Z
Request 11 window start: 2019-10-30T23:19:30Z window end: 2019-10-30T23:40:30Z
Request 12 window start: 2019-10-30T23:40:30Z window end: 2019-10-31T00:01:30Z
Request 13 window start: 2019-1

In [38]:
# Submit the fully formed RequestGroup
response = requests.post(
    'https://observe.lco.global/api/requestgroups/validate/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure the API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('API call failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()  # The API will return the newly submitted requestgroup as json

print('Total number of observing hours needed for this request: ',
      requestgroup_dict['request_durations']['duration']/60./60)

Total number of observing hours needed for this request:  2.412222222222222


# Do not run the following cell until you have confirmed with an instructor your observation.

In [ ]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure this API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()

# Print out the url on the portal where we can view the submitted request
print('View this observing request: https://observe.lco.global/requestgroups/{}/'.format(requestgroup_dict['id']))

print('This request was successfully submitted at {}'.format(datetime.datetime.now()))

In [3]:
TARGET_PERIOD = 7.6 # hours
cadence_phase = (1/2./3.)/3.7
print(cadence_phase)
print(TARGET_PERIOD*cadence_phase*60,'minutes')

0.04504504504504504
20.540540540540537 minutes
